# Logistics
## Problem Overview

Kaggle competition link [here](https://www.kaggle.com/c/open-shopee-code-league-logistic).

### Logistics Performance
Due to the recent COVID-19 pandemic across the globe, many individuals are increasingly turning to online platforms like Shopee to purchase their daily necessities. This surge in online orders has placed a strain onto Shopee and our logistics providers but customer expectations on the timely delivery of their goods remain high. On-time delivery is arguably one of the most important factors of success in the eCommerce industry and now more than ever, we need to ensure the orders reach our buyers on time in order to build our users’ confidence in us.

In order to handle the millions of parcels that need to be delivered everyday, we have engaged multiple logistics providers across the region. Only the best logistics providers that are able to meet Shopee’s delivery standards are partnered with us.

The performance of these providers is monitored regularly and each provider is held accountable based on the Service Level Agreements (SLA). Late deliveries are flagged out and penalties are imposed on the providers to ensure they perform their utmost.

The consistent monitoring and process of holding our logistics providers accountable allows us to maintain our promise of timely deliveries to our buyers.

### Task
Identify all the orders that are considered late depending on the Service Level Agreements (SLA) with our Logistics Provider.

For the purpose of this question, assume that all deliveries are considered successful by the second attempt.

### Basic Concepts
- Each orderid represents a distinct transaction on Shopee.
- SLA can vary across each route (A route is defined as Seller’s Location to Buyer’s Location) - Refer to SLA_matrix.xlsx
- Pick Up Time is defined as the time when the 3PL picks up the parcel and begins to process for delivery. It marks the start of the SLA calculation.
- Delivery Attempt is defined as an attempt made by the 3PL to deliver the parcel to the customer. It may or may not be delivered successfully. In the case when it is unsuccessful, a 2nd attempt will be made. A parcel that has no 2nd attempt is deemed to have been successfully delivered on the 1st attempt.
- All time formats are stored in epoch time based on Local Time (GMT+8).
- Only consider the date when determining if the order is late; ignore the time.
- Working Days are defined as Mon - Sat, Excluding Public Holidays.
- SLA calculation begins from the next day after pickup (Day 0 = Day of Pickup; Day 1 = Next Day after Pickup)

2nd Attempt must be no later than 3 working days after the 1st Attempt, regardless of origin to destination route (Day 0 = Day of 1st Attempt; Day 1 = Next Day after 1st Attempt).  

Only consider the date when determining if the order is late; ignore the time.  

Assume the following Public Holidays: 

1. 2020-03-08 (Sunday);
2. 2020-03-25 (Wednesday);
3. 2020-03-30 (Monday);
4. 2020-03-31 (Tuesday)

### Submission Format
Check each delivery order and determine whether it is late.

Two columns required:

- orderid.
- is_late: assign value 1 if the order is late, otherwise 0.

Your submission should have 3,176,313 rows (excluding headers), each with 2 columns.

### Tips:
1) You are advised to run your tests on a sample of the dataset first.

2) If you are unable to solve the entire problem within the time limit, create the output csv with the required number of columns and rows based on a subset of the problem first.

### Import Libraries

In [10]:
import pandas as pd
import datetime
import time
import numpy as np

### Read the File

In [2]:
df = pd.read_csv(r'C:\Users\angel\Documents\Shopee Code League\Logistics\delivery_orders_march.csv')

### Clean Data Types

- convert times to datetime, in local time (+8 GMT)
- fill na as 0 for 2nd delivery attempt since this can be null and it needs to be filled to convert it to datetime
- then convert to pandas datetime to extract the dates only (easier when working with dataframe)

In [3]:
# time_zone = 28800 

# Use localtime
df['pick'] = df['pick'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(x))))
df['1st_deliver_attempt'] = df['1st_deliver_attempt'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(x))))
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].fillna(0).astype(int)
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(x))))


# To datetime
# df['pick'] = df['pick'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
# df['1st_deliver_attempt'] = df['1st_deliver_attempt'].astype(int)
# df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].fillna(0).astype(int)
# df['1st_deliver_attempt'] = df['1st_deliver_attempt'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
# df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

# To pandas datetime
df['pick'] = pd.to_datetime(df['pick'], format = '%Y-%m-%d %H:%M:%S').dt.date
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'], format = '%Y-%m-%d %H:%M:%S').dt.date
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'], format = '%Y-%m-%d %H:%M:%S').dt.date
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


Extract the buyer and seller address since SLA matters with island origin and destination. This is available as the *last* word in the address columns.

In [4]:
df['b_islands'] = df['buyeraddress'].apply(lambda x: x.split(' ')[-1].lower())
df['s_islands'] = df['selleraddress'].apply(lambda x: x.split(' ')[-1].lower())

df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,b_islands,s_islands
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,luzon,manila


### Create date dictionary

Since we're trying to extract the number of days from pickup date to delivery date, we will create a dates dictionary that spans the start and end dates of our dataset. Then, we will map the dates to numbers, which corresponds to its relative position from the start of our date dictionary.

First, we specify the start and end date. This is done by selecting the minimum day of pickup and maximum day of delivery.

In [5]:
start_date = df['pick'].min()
end_date = df['2nd_deliver_attempt'].max()

Then, we create a date range from the dates earlier.

In [6]:
dates = pd.date_range(start = start_date , end = end_date).date

We convert this date range to a list of dates.

In [7]:
dates = list(dates)

We remove the holidays since these are not part of SLA.

In [11]:
# holidays

holidays = [datetime.date(2020, 3, 8), datetime.date(2020, 3, 25), datetime.date(2020, 3, 30), datetime.date(2020, 3, 31)]
for day in holidays:
    dates.remove(day)

We create a dataframe with the list of dates from earlier, with corresponding indices assigned.

In [12]:
datedf = pd.DataFrame(data = dates, index = np.arange(start = 0, stop = len(dates)), columns = ['date'])

We extract the day of week from the date column in our dataframe.

In [13]:
datedf['dayofweek'] = pd.to_datetime(datedf['date']).dt.dayofweek

We only select the dates less than 6 since 6 corresponds to a Sunday. Sundays are also not counted in the SLA.

In [14]:
finaldates = datedf[datedf['dayofweek'] < 6]

In [15]:
fdates = list(finaldates['date'])

We create a dictionary from the list of final dates created earlier. They are assigned to a day count.

In [16]:
dictdate = dict(zip(fdates, list(range(1, len(fdates)))))

### Add SLA column

In [17]:
def sla(row):
    if row['s_islands'] == 'manila' and row['b_islands'] == 'manila':
        return 3
    elif row['s_islands'] == 'manila' and row['b_islands'] == 'luzon':
        return 5
    elif row['s_islands'] == 'luzon' and row['b_islands'] == 'luzon':
        return 5
    else:
        return 7
    
df['sla'] = df.apply(sla, axis=1)

In [18]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,b_islands,s_islands,sla
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila,3
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,luzon,manila,5


### Map dictionary of dates

In [19]:
df['pup'] = df['pick'].map(dictdate)
df['1stdel'] = df['1st_deliver_attempt'].map(dictdate)
df['2nddel'] = df['2nd_deliver_attempt'].map(dictdate)

In [20]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,b_islands,s_islands,sla,pup,1stdel,2nddel
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila,3,1,4,NaN
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila,3,3,5,8.0
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila,3,3,5,NaN
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila,3,4,6,NaN
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,luzon,manila,5,3,5,NaN


### Get difference

In [21]:
df['1stdiff'] = df['1stdel'] - df['pup']
df['2nddiff'] = df['2nddel'] - df['1stdel']

# 1st delivery late
df['1stlate'] = df['1stdiff'] > df['sla'] #Count starts next day

# 2nd delivery late
df['2nddel'] = df['2nddel'].fillna(0)
df['2ndlate'] = df['2nddiff'] > 3

# Get is_late column
df['is_late'] = df['1stlate'] + df['2ndlate']
df['is_late'] = df['is_late'].astype(int)

C:\Users\angel\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [22]:
df.head(20)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,b_islands,s_islands,sla,pup,1stdel,2nddel,1stdiff,2nddiff,1stlate,2ndlate,is_late
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila,3,1,4,0.0,3,NaN,False,False,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila,3,3,5,8.0,2,3.0,False,False,0
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila,3,3,5,0.0,2,NaN,False,False,0
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila,3,4,6,0.0,2,NaN,False,False,0
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,luzon,manila,5,3,5,0.0,2,NaN,False,False,0
5,2222597288,2020-03-04,2020-03-07,1970-01-01,C-846 park&shop opens Valeda/Sunrise) CompuWar...,33 sta.mesa 1929 hawthorn Metro Manila,manila,manila,3,3,6,0.0,3,NaN,False,False,0
6,2222738456,2020-03-02,2020-03-05,2020-03-09,"Ka-insong stockholme Lhuilier, polanco. malen ...","salcedo Midlands 980 WALL, 16C paraiso Mall（Ph...",manila,manila,3,1,4,7.0,3,3.0,False,False,0
7,2224695304,2020-03-02,2020-03-10,1970-01-01,Reel zambo.ciyy puyat.cor road/infront #129A L...,Polar 75 mulawinan 1G-3 señora Text/Call 2P-01...,manila,manila,3,1,8,0.0,7,NaN,True,False,1
8,2224704587,2020-03-04,2020-03-05,2020-03-09,"Lumber,Naprada POT kadir Bagyan blanga FIRENDS...",skies dept. Crystal apello Stockroom.Star (DAG...,luzon,manila,5,3,4,7.0,1,3.0,False,False,0
9,2225138267,2020-03-04,2020-03-10,1970-01-01,"Hengtong Buhaynasapa, Lask Southville8A Colgat...",9x Comfort Greenheights Lifehomes AANI Metro M...,visayas,manila,7,3,8,0.0,5,NaN,False,False,0


### Export output

In [23]:
output = df[['orderid', 'is_late']]
output.to_csv('localtime.csv', index = False)